### Operational Space Control (Khatib)

1. The controller is defined as $\Gamma = \sum_{k=1}^N J_{k|prec(k)}^T F_{k|prec(k)}$
2. The prioritized jacobian for task $k$ is given by $J_{k|prec(k)} := J_k N_{prec(k)}$
3. $N_{prec(k)} = I - \sum_{i=1}^{k-1} \bar J_{i|prec(i)} J_{i|prec(i)}$
4. $N_{prec(k)} = N_{k-1|prec(k-1)} N_{k-2|prec(k-2)} ... I$. See *sentis A.24*. This is equivalent to $N_{prec(k)} = N_{k-1|prec(k-1)} N_{prec(k-1)}$

Algorithm
1. Initialize $N_{prec(1)} = I$, $priority = 1$
2. $N_{1|prec(1)} \leftarrow (I-\bar J_1 J_1)$
3. For $t_i \in T$
<br> a) If $priority(t_i) > priority$,
<br> $$N_{prec(i)} = N_{i-1|prec(i-1)} N_{prec(i-1)}$$
<br> $$priority = priority(t_i)$$ 
<br> b) Set $N_{prec(i)}$ in current task
<br> c) Calculate $J_{i|prec(i)} := J_i N_{prec(i)}$
<br> d) Calculate $N_{i|prec(i)} := (I-\bar J_{i|prec(i)} J_{i|prec(i)})$

### Hierarchical Quadratic Programming (Mansard)

1. The heart of operational space control is the implicit constrained minimization of the kinetic energy that is embedded within the dynamically consistent jacobian inverse $\bar J$. We can appreciate this more clearly with the use of lagrange multipliers applied to the following problem: $$\min_{\dot q} \frac{1}{2} \dot q^T M \dot q$$ $$st. \dot x = J \dot q$$
2. The lagrangian is given by $L = \dot q^T M \dot q - \lambda^T (\dot x - J \dot q)$ giving the partial derivatives $\frac{\partial L}{\partial \dot q} = M \dot q - J^T \lambda = 0$ and the trivial constraint equation $\frac{\partial L}{\partial \dot \lambda} = \dot x - J \dot q = 0$. Premultiplying the first equation by $J$ and substituting the second, we have $J \dot q = (J M^-1 J^T) \lambda = \Lambda^{-1} \lambda \rightarrow \lambda = \Lambda \dot x$. Putting this result back to the first equation, we have $\dot q = M^{-1} J^T \Lambda \dot x = \bar J \dot x$.

### Prioritized Motion-Force Control TSID
1. 

### Optimization-based robot control (Andrea Del Prete)
1. Inverse dynamics control can be expressed as a constrained optimization problem to include limits:
$$\min_{\tau,\ddot q} ||\ddot q - \ddot q_d||^2$$
$$s.t. M \ddot q + h = \tau$$
$$\tau_{min} \leq \tau \leq \tau_{max}$$
$$i_{min} \leq k_{\tau} \tau \leq i_{max}$$

2. Since torques and accelerations are optimization variables in least squares program, we cannot include velocity and position constraints. To do so, we can assume constant acceleration and have:
$$v_{min} \leq v + \Delta t \dot v \leq v_{max}$$
$$q_{min} \leq q + \Delta t \dot v + \frac{1}{2} \dot v \Delta t^2 \leq q_{max}$$

3. The position limits can lead to high accelerations as especially in the limits and is typically incompatible with torque/current limits.

4. For task space, error dynamics need to be expressed as affine functions, for example, for end-effector positions, we have:
$$\ddot e = -K_p e - K_d \dot e$$
$$\ddot x - \ddot x_d = -K_p e - K_d \dot e$$
$$J \ddot q + \dot J \dot q - \ddot x_d = -K_p e - K_d \dot e$$
$$J \ddot q =- \dot J \dot q + \ddot x_d -K_p e - K_d \dot e \Longrightarrow A_q \ddot q = a$$

5. Generalizing, we have an additional linear constraint:
$$g(y) := \begin{bmatrix} A_q & A_u \end{bmatrix} \begin{bmatrix} \ddot q \\ u \end{bmatrix} - a \Longrightarrow Ay - a$$

6. In task space, our optimization problem leads to:
$$\min_{y = (\ddot q, \tau)} ||Ay - a||^2$$
$$s.t. \begin{bmatrix} M & -S^T \end{bmatrix} y = -h$$

7. Integrating a soft contact, use an estimated $\hat f$:
$$\min_{y = (\ddot q, \tau)} ||Ay - a||^2$$
$$s.t. \begin{bmatrix} M & -S^T \end{bmatrix} y = -h + J^T \hat f$$
This only works if the contact is actually soft, otherwise the mismatch can lead to numerical stability issues.

8. Integrating rigid contact as a constraint can be done by specifying the constraint manifold $c(q) = constant$, i.e. that contacts do not move. Thus, $J_c \dot q = 0$ and $J_c \ddot q + \dot J_c \dot q = 0$. Incorporating this into our optimization problem:
$$\min_{y = (\ddot q, f, \tau)} ||Ay - a||^2$$
$$s.t. \begin{bmatrix} J_c & 0 & 0 \\ M & -J^T & -S^T \end{bmatrix} y = \begin{bmatrix} -J_c \ddot q \\ -h \end{bmatrix}$$

9. For multi-task control, a hierarchical LSP can be used where each lower priority task (with priority $i$) will have an additional constraint $g_j (y) = g_j^*, \forall j < i$

10. Computational cost is always dominated by the Hessian (Cholesky decomposition) which taks $O(n^3)$